## Part 3 - Monitoring roads

#### Imports

In [49]:
import pandas as pd
import numpy as np
import folium
import sys
from IPython.display import display
import ipywidgets as widgets

### Load data

#### Accidents data

In [50]:
def load_dataset():  
    veh_cols = ['Num_Acc', 'num_veh', 'senc', 'catv', 'obs', 'obsm', 'choc']
    usa_cols = ['Num_Acc', 'num_veh', 'place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu', 'locp']
    lie_cols = ['Num_Acc', 'catr', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ']
    car_cols = ['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'atm', 'col', 'gps', 'lat', 'long']

    veh = pd.read_csv('../Accidents/vehicules_2005.csv', usecols=veh_cols)
    usa = pd.read_csv('../Accidents/usagers_2005.csv', usecols=usa_cols)
    lie = pd.read_csv('../Accidents/lieux_2005.csv', usecols=lie_cols)
    car = pd.read_csv('../Accidents/caracteristiques_2005.csv', encoding='latin-1', usecols=car_cols)

    for i in range(2006, 2018, 1):
        lie = pd.concat([lie, pd.read_csv('../Accidents/lieux_' + str(i) + '.csv', usecols=lie_cols)], axis=0)
        usa = pd.concat([usa, pd.read_csv('../Accidents/usagers_' + str(i) + '.csv', usecols=usa_cols)], axis=0)
        veh = pd.concat([veh, pd.read_csv('../Accidents/vehicules_' + str(i) + '.csv', usecols=veh_cols)], axis=0)
        try:
            car = pd.concat([car, pd.read_csv('../Accidents/caracteristiques_' + str(i) + '.csv',
                                              encoding='latin-1', usecols=car_cols)], axis=0)
        except:
            car = pd.concat(
                [car, pd.read_csv('../Accidents/caracteristiques_' + str(i) + '.csv', usecols=car_cols,
                                  encoding='latin-1', sep='\t')], axis=0)
            
    return veh, usa, car, lie

veh, usa, car, lie = load_dataset()

car.head()

Num_Acc  an  mois  jour  hrmn  lum  agg  atm  col    com gps  \
0  200500000001   5     1    12  1900    3    2  1.0  3.0   11.0   M   
1  200500000002   5     1    21  1600    1    2  1.0  1.0   51.0   M   
2  200500000003   5     1    21  1845    3    1  2.0  1.0   51.0   M   
3  200500000004   5     1     4  1615    1    1  1.0  5.0   82.0   M   
4  200500000005   5     1    10  1945    3    1  3.0  6.0  478.0   M   

         lat    long  dep  
0  5051500.0  294400  590  
1  5053700.0  280200  590  
2  5054600.0  280000  590  
3  5098700.0  240800  590  
4  5096400.0  247500  590

#### Correspondance data

With this table, we have a correspondance between communal code in our accidents dataset, and the postal code, which is more convenient to identify a city.

In [51]:
postal_1 = pd.read_csv('../Accidents/postal_part1.csv')
postal_2 = pd.read_csv('../Accidents/postal_part2.csv')

com_postal_pivot = pd.concat([postal_1, postal_2], ignore_index=True).set_index('Code_postal')
com_postal_pivot.head(5)

Geo Point  CODE_COM INSEE_COM     NOM_COM  \
Code_postal                                                                 
95450.0        49.0848358031,1.9748748362         2     95002    ABLEIGES   
76280.0      49.6097472297,0.241445095242       716     76716    TURRETOT   
76690.0       49.6093542995,1.07016467145       105     76105  LE BOCASSE   
10290.0       48.3605004241,3.59036233813        54     10054   BOURDENAY   
51220.0        49.345648783,3.97511257054       329     51329      LOIVRE   

             SUPERFICIE  POPULATION CODE_DEPT        NOM_DEPT  CODE_REG  \
Code_postal                                                               
95450.0           792.0         933        95      VAL-D'OISE        11   
76280.0           612.0        1511        76  SEINE-MARITIME        28   
76690.0           860.0         707        76  SEINE-MARITIME        28   
10290.0          1856.0         128        10            AUBE        44   
51220.0          1024.0        1197        51           MARNE        44   

                                       NOM_REG Libelle_acheminement  \
Code_postal                                                           
95450.0                          ILE-DE-FRANCE             ABLEIGES   
76280.0                              NORMANDIE             TURRETOT   
76690.0                              NORMANDIE           LE BOCASSE   
10290.0      ALSACE-CHAMPAGNE-ARDENNE-LORRAINE            BOURDENAY   
51220.0      ALSACE-CHAMPAGNE-ARDENNE-LORRAINE               LOIVRE   

            Nom_commune  
Code_postal              
95450.0        ABLEIGES  
76280.0        TURRETOT  
76690.0      LE BOCASSE  
10290.0       BOURDENAY  
51220.0          LOIVRE

## Filtering on postal code

In [52]:
def postal_code_selection(code = None):
    
    # Handle given postal code (optional)
    if isinstance(code, str): 
        postal_code_input = code
    else: 
        postal_code_input = input("Select a postal code : ")
        
    try:
        result = com_postal_pivot.loc[int(postal_code_input) , : ]
    except:
        print('Postal code not found in database')

    if isinstance(result, pd.Series):
        #print("Single line result")
        return result
    elif isinstance(result, pd.DataFrame):
        #print("Multi line result") Get first row
        return result.iloc[0]
    else:
        print(result.shape)

postal_code_selection('75015')

Geo Point               48.840040811,2.29268840406
CODE_COM                                       115
INSEE_COM                                    75115
NOM_COM                   PARIS-15E-ARRONDISSEMENT
SUPERFICIE                                     846
POPULATION                                  238190
CODE_DEPT                                       75
NOM_DEPT                                     PARIS
CODE_REG                                        11
NOM_REG                              ILE-DE-FRANCE
Libelle_acheminement                         PARIS
Nom_commune                               PARIS 15
Name: 75015.0, dtype: object

## Map drawing

In [53]:
def export_map(
    scale='Country',
    post_code='75013', 
    department='75',
    veh_type='all', 
    weather='all', 
    luminosity='all', 
    gravity='all',
    usa_sexe='all',
    export_name='plot_map.html',
    begin='10', end='18'):

    # Cross data - Filter on Dpt & Com
    if scale == 'City':
        result = postal_code_selection(post_code)
        display_scale = 14
        car_tmp = car[(car['dep'] == int(result['CODE_DEPT']+'0')) & (car['com'] == result['CODE_COM'])]
        centered_location = [float(result["Geo Point"].split(",")[0]), float(result["Geo Point"].split(",")[1])]
    elif scale == 'Department':
        car_tmp = car[(car['dep'] == int(str(department) + '0'))]
        if department == '75':
            display_scale = 12
            centered_location = [48.866997, 2.339459]
        else:
            display_scale = 9
            result = postal_code_selection(department+'000')
            centered_location = [float(result["Geo Point"].split(",")[0]), float(result["Geo Point"].split(",")[1])]
    elif scale == 'Country':
        car_tmp = car
        display_scale = 6
        centered_location = [47.12, 2.5]
    
    zone_df = pd.merge(veh, car_tmp, on='Num_Acc')
    zone_df = pd.merge(zone_df, lie, on='Num_Acc')
    zone_df = zone_df[zone_df.gps == 'M']
    
    # Filter on road type
    #if scale == 'Country':
        #zone_df = zone_df[zone_df.catr == 1.0]

    # Filter empty coordinates
    zone_df = zone_df[(zone_df.long != 0) & (zone_df.lat != 0)]
    zone_df = zone_df[(zone_df.long != 0.0) & (zone_df.lat != 0.0)]
    zone_df = zone_df.dropna(subset=['lat', 'long'])
    
    # Filter on vehicle type
    if (veh_type == 'Cars'):
        zone_df = zone_df[(zone_df.catv == 7)]
    elif veh_type == 'Motorcycles':
        zone_df = zone_df[(zone_df.catv == 2) | (zone_df.catv == 30) | (zone_df.catv == 31) | 
                          (zone_df.catv == 32) | (zone_df.catv == 33) | (zone_df.catv == 34)]
    elif veh_type == 'Bikes':
        zone_df = zone_df[(zone_df.catv == 1)]
    elif veh_type == 'Public transports':
        zone_df = zone_df[(zone_df.catv == 37) | (zone_df.catv == 38) | (zone_df.catv == 39) | 
                          (zone_df.catv == 40)]
        
    # Luminosity conditions
    if (luminosity == 'Day'):                   zone_df = zone_df[(zone_df.lum == 1)]
    elif luminosity == 'Dusk/Dawn':             zone_df = zone_df[(zone_df.lum == 2)]
    elif luminosity == 'Night (with lighting)': zone_df = zone_df[(zone_df.lum == 5)]
    elif luminosity == 'Night (no lighting)':   zone_df = zone_df[(zone_df.lum == 3) | (zone_df.lum == 4)]
    
    # Atmospheric conditions
    if weather == 'Normal/Sunny':   zone_df = zone_df[(zone_df.atm == 1) | (zone_df.atm == 7)]
    elif weather == 'Rain':         zone_df = zone_df[(zone_df.atm == 2) | (zone_df.atm == 3)]
    elif weather == 'Snow':         zone_df = zone_df[(zone_df.atm == 4)]
    elif weather == 'Fog':          zone_df = zone_df[(zone_df.atm == 5)]
        
    # Filter on time interval
    zone_df = zone_df[(zone_df.an >= int(begin)) & (zone_df.an <= int(end))]
    zone_df = pd.merge(zone_df, usa, on='Num_Acc')
    
    # Coordinates conversion
    zone_df['lat'] = zone_df['lat'].replace('-', 0).fillna(0).astype('float') / 100000
    zone_df['long'] = zone_df['long'].replace('-', 0).fillna(0).astype('float') / 100000
    zone_df = zone_df[zone_df.long != 0.0]
    #print(zone_df.shape[0], ' rows with good gps cordinates.\n')

    return zone_df

zone_df = export_map()

In [54]:
zone_df['dep'] = zone_df['dep'].astype(str)
zone_df = zone_df[zone_df['lat'] > 40]
zone_df['lat'] = zone_df['lat'] - np.mean(zone_df['lat'])

In [55]:
zone_df.columns

Index(['Num_Acc', 'senc', 'catv', 'obs', 'obsm', 'choc', 'num_veh_x', 'an',
       'mois', 'jour', 'hrmn', 'lum', 'agg', 'atm', 'col', 'com', 'gps', 'lat',
       'long', 'dep', 'catr', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'lartpc',
       'larrout', 'surf', 'infra', 'situ', 'place', 'catu', 'grav', 'sexe',
       'trajet', 'secu', 'locp', 'an_nais', 'num_veh_y'],
      dtype='object')

In [56]:
def genre_proc(genre):
    if str(genre) == '1': return 'male'
    else: return 'female'
    
    
def vehicle_proc(vehicle):
    if str(vehicle) in ['3', '7', '8', '9']: return 'Voiture'
    elif str(vehicle) in ['33', '34', '2', '30', '31', '32']: return 'Moto'
    elif str(vehicle) in ['1']: return 'Velo'
    elif str(vehicle) in ['10', '13', '14', '20', '21', '16', '17']: return 'Camion'
    
    
def atm_proc(weather):
    if str(weather) in ['2.0', '3.0', '5.0']:        return 'Rain'
    elif str(weather) in ['4.0', '6.0']:     return 'Snow, Storm'
    elif str(weather) in ['1.0', '8.0', '7.0']: return 'Normal'
    
zone_df['catv_str'] = zone_df['catv'].apply(lambda x: vehicle_proc(x))
zone_df['sexe_str'] = zone_df['sexe'].apply(lambda x: genre_proc(x))
zone_df['atm_str'] = zone_df['atm'].apply(lambda x: atm_proc(x))

In [57]:
def treat_year(an) :
    return int(str(20) + str(an))

zone_df['annee'] = zone_df['an'].apply(lambda x: treat_year(x))
zone_df['age'] = zone_df['annee'] - zone_df['an_nais']

In [58]:
zone_df = zone_df.dropna()

In [59]:
zone_df = zone_df[zone_df['larrout'] < 200]

In [60]:
zone_df = zone_df[zone_df['larrout'] > 0]

In [104]:
def modif_grav(grav) :
    if grav == 1 :
        return 1
    elif grav == 3 :
        return 3
    elif grav == 4 :
        return 2
    else :
        return 4
zone_df['grav'] = zone_df['grav'].apply(lambda x : modif_grav(x))

In [105]:
def modif_grav(grav) :
    if grav == 1 :
        return "Indemne"
    elif grav == 2 :
        return "Blessé léger"
    elif grav == 3 :
        return  "Blessé hosp."
    else :
        return "Mort"
zone_df['grav_str'] = zone_df['grav'].apply(lambda x : modif_grav(x))

In [65]:
zone_df = zone_df[zone_df['lartpc'] < 40]

In [85]:
zone_df.to_csv('zone_df.csv')

## Interactive Map

In [120]:
import altair as alt

def plot_map(df) :

    #df = df[df['catr']==4]
    
    if len(df) >= 5000 :
        df = df.sample(30000)

    scales = alt.selection_interval(bind='scales')

    selection = alt.selection_multi(fields=['grav'])

    color = alt.Color('grav:Q', scale=alt.Scale(scheme='redblue', domain=[5,0]), legend=None)

    brush = alt.selection(type='interval')

    ### MAP
    map = alt.Chart(df).mark_circle(size=10).encode(
        x='long',
        y=alt.Y('lat'),
        color=color,
        tooltip=[
            alt.Tooltip('hrmn:Q', title="Heure"),
            alt.Tooltip('catv_str:N', title="Veh. Cat."),
            alt.Tooltip('grav_str:N', title="Gravity"),
            alt.Tooltip('sexe_str:N', title="Casualty genre")
        ]
    ).properties(
        width=275,
        height=275
    ).add_selection(
        brush
    )
    
    ## LEFT - RIGHT
    bars_0 = alt.Chart(df).mark_bar().encode(
        alt.X("grav_str:N", axis=alt.Axis(title='Gravité')),
        y='count()',
        color=color,
        tooltip=[
            alt.Tooltip('grav_str:N', title="Gravité")]
    ).transform_filter(
        brush
    ).properties(
        width=275,
        height=150
    )

    bars_1 = alt.Chart(df).mark_bar().encode(
        alt.X("hrmn:Q", axis=alt.Axis(title='Heure')),
        y='count()',
        color=color,
        tooltip=[
            alt.Tooltip('hrmn:Q', title="Heure"),
            alt.Tooltip('grav_str:N', title="Gravité")]
    ).transform_filter(
        brush
    ).properties(
        width=275,
        height=150
    )

    
    bars_2 = alt.Chart(df).mark_bar().encode(
        alt.X("sexe_str:N", axis=alt.Axis(title='Sexe')),
        y='count()',
        color=color,
        tooltip=[
            alt.Tooltip('sexe_str:N', title="Sexe"),
            alt.Tooltip('grav_str:N', title="Gravité")]
    ).transform_filter(
        brush
    ).properties(
        width=275,
        height=150
    )

    
    bars_3 = alt.Chart(df).mark_bar().encode(
        alt.X("catv_str:N", axis=alt.Axis(title='Cat. Véhicule')),
        y='count()',
        color=color,
        tooltip=[
            alt.Tooltip('catv_str:N', title="Cat. Véhicule"),
            alt.Tooltip('grav_str:N', title="Gravité")]
    ).transform_filter(
        brush
    ).properties(
        width=275,
        height=150
    )

    bars_4 = alt.Chart(df).mark_bar().encode(
        alt.X("lartpc:Q", bin=alt.Bin(maxbins=50), axis=alt.Axis(title='Largeur TPC')),
        y='count()',
        color=color,
        tooltip=[
            alt.Tooltip('lartpc:Q', title="Largeur TPC"),
            alt.Tooltip('grav_str:N', title="Gravité")]
    ).transform_filter(
        brush
    ).properties(
        width=275,
        height=150
    )

    
    bars_5 = alt.Chart(df).mark_bar().encode(
        alt.X("atm_str:N", axis=alt.Axis(title='Météo')),
        y='count()',
        color=color,
        tooltip=[
            alt.Tooltip('atm_str:N', title="Météo"),
            alt.Tooltip('grav_str:N', title="Gravité")]
    ).transform_filter(
        brush
    ).properties(
        width=275,
        height=150
    )
    
    
    bars_left = alt.Chart(df).mark_bar().encode(
        alt.X("annee:Q", axis=alt.Axis(title='Année')),
        y='count()',
        color=color,
        tooltip=[
            alt.Tooltip('annee:Q', title="Année"),
            alt.Tooltip('grav_str:N', title="Gravité")]
    ).transform_filter(
        brush
    ).properties(
        width=275,
        height=275
    )


    bars_right = alt.Chart(df).mark_bar().encode(
        alt.X("age:Q", bin=alt.Bin(maxbins=50), axis=alt.Axis(title='Age')),
        y='count()',
        color=color,
        tooltip=[
            alt.Tooltip('age:Q', title="Age"),
            alt.Tooltip('grav_str:N', title="Gravité")]
    ).transform_filter(
        brush
    ).properties(
        width=275,
        height=275
    )

    chart = ( bars_0 | bars_1 | bars_2 ) & (bars_left | map | bars_right) & ( bars_3 | bars_4 | bars_5 )

    return chart.save('all_6.html')


In [121]:
plot_map(zone_df)